In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import matplotlib.pyplot as plt
import math

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
def create_data(start, end, step):
    steps = torch.arange(start, end, step)
    tensor = torch.tensor([], dtype=torch.float64)
    for step in steps:
        tensor = torch.cat((tensor, torch.tensor([step * math.sin(step**2 + 2 + math.exp(step))], dtype=torch.float64)))
    return steps.unsqueeze(dim=1), tensor.unsqueeze(dim=1)
X, y = create_data(0, 1, 0.02)
    

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
def create_plot(train_data = X_train,
                train_labels = y_train,
                test_data = X_test,
                test_labels = y_test,
                predictions=None):
    plt.figure(figsize=(10, 8))
    plt.scatter(X_train, y_train, color='b')
    plt.scatter(X_test, y_test, color='r')
    if predictions is not None:
        plt.scatter(predictions[0], predictions[1], color='g')
    plt.show()
create_plot()

In [ ]:
class TorchLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.cos_coef = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.exponent = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.sin(self.cos_coef * x + torch.exp(self.exponent))
        

In [ ]:
torch.manual_seed(24212)
model = TorchLinearRegression()

model.parameters()

In [ ]:
with torch.inference_mode(): 
    y_preds = model(X_test)

create_plot(predictions=(X_test, y_preds))
model.train()

In [ ]:
with torch.inference_mode(): 
    y_preds = model(X_test)

create_plot(predictions=(X_test, y_preds))

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=0.1)


In [ ]:
epochs = 100
for epoch in range(epochs):
    ### Training

    # Put model in training mode (this is the default state of a model)
    model.train()

    # 1. Forward pass on train data using the forward() method inside 
    y_pred = model(X_train).float()
    # print(y_pred)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, y_train.float())
    # 3. Zero grad of the optimizer
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    ### Testing
    with torch.inference_mode():
        if epoch % 10 == 0:
            y_test_preds = model(X_test)
            score = loss_fn(y_test_preds, y_test)
            print(f'Epoch: {epoch}, Loss: {loss}, Test loss: {score}')
            print(model.state_dict())
    # Put the model in evaluation mode
    model.eval()

In [ ]:
list(model.parameters())

In [ ]:
from sklearn.datasets import make_circles

In [ ]:
n = 1000

In [ ]:
X, y = make_circles(n)

In [ ]:
X = torch.tensor(X)

In [ ]:
y = torch.tensor(y)

In [ ]:
import pandas as pd

In [ ]:
data = pd.DataFrame({'X': X[:, 0], 'Y': X[:, 1], 'labels': y})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train = X_train.type(torch.float)
X_test = X_test.type(torch.float)
y_train = y_train.type(torch.float)
y_test = y_test.type(torch.float)

In [ ]:
X_train

In [ ]:
class CirclesClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 5)
        self.relu = nn.ReLU6()
        self.layer2 = nn.Linear(5, 1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x
    
torch.manual_seed(234622)
model_1 = CirclesClassifier()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.2)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)

In [ ]:
epochs = 1000

for epoch in range(epochs):
    model_1.train()
    y_preds = model_1(X_train).squeeze(dim=1)
    optimizer.zero_grad()

    loss = loss_fn(y_preds, y_train)
    loss.backward()
    optimizer.step()
    
    with torch.inference_mode():
        if epoch % 10 == 0:
            y_test_preds = model_1(X_test).squeeze(dim=1)
            score = loss_fn(y_test_preds, y_test)
            print(f'Epoch: {epoch}, Loss: {loss}, Test loss: {score}')
    # Put the model in evaluation mode
    model_1.eval()
    
    

In [ ]:
y_preds = torch.round(torch.sigmoid(model_1(X_test).squeeze(dim=1))).detach().numpy()

In [ ]:
y_preds

In [ ]:
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test)

In [ ]:
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_preds)

In [ ]:
model_1_built = nn.Sequential(nn.Linear(2, 5), nn.Linear(5, 1), nn.Sigmoid())


In [ ]:
import torchvision
from torchvision import datasets

In [ ]:
train_data = datasets.FashionMNIST(
    train=True,
    root='data',
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((28, 28))
    ])
)


In [ ]:
train_data[0]

In [ ]:
class_names = train_data.classes

In [ ]:
class_names

In [ ]:
class_idx_dict = train_data.class_to_idx
print(class_idx_dict)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
pic, idx = train_data[0]
pic = pic.squeeze()
plt.imshow(pic)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_batches = DataLoader(
    dataset = train_data,
    batch_size=32,
    shuffle=True)

In [ ]:
train_features_batch, train_features_batch_index = next(iter(train_batches))